# Teste A|B, A|B Testing, Split Testing
Teste A|B é um teste  que divide o acesso a uma página online ou produto em duas versões diferentes, A(controle) e B(experimento), e depois mede-se a taxa de conversão que podem ser  vendas, clicks, preferência, compartilhamentos entre outras. As diferenças nas páginas/produtos podem ser em cores, botões, chamadas para ação ou serem totalmente diferentes mas com o mesmo objetivo. 

Suas aplicações são comuns em e-commerce e anúncios como Google Ads, Redes Sociais, email marketing e landing pages. Também é usado na indústria para lançamento de versões produtos para avaliar cores, tamanho e forma.. Mesmo com aparência simples não é fácil encontrar variáveis estatísticamente relevantes  para o experimento, isto é, é necessário grande quantidade de dados e tempo para rodar o teste, neste artigo melhor detalhado o uso dos Testes A|B:[O que é Teste A|B.](https://resultadosdigitais.com.br/blog/o-que-e-teste-ab/)

Embora possa parecer simples deve ser cuidadosamente projetado para obter resultados conclusivos e evitar enviesamento como descrito nest artigo em inglês [7 A/B Testing Questions and Answers in Data Science Interviews](http://https://towardsdatascience.com/7-a-b-testing-questions-and-answers-in-data-science-interviews-eee6428a8b63), que faz um apanhado da teoria do Teste A|B como questões de entrevista de emprego.


- Para aprofundar no assunto super recomendo o podcast hipsters tech com o time da NetShoes e o material da página. Você pode acessar [aqui.](https://hipsters.tech/testes-ab-hipsters-59/).

- [Experiments at Airbnb](https://medium.com/airbnb-engineering/experiments-at-airbnb-e2db3abf39e7) - Um estudo de caso do Airbnb que mostra as armadilhas do Teste A|B e como evitar.
- [What is A|B Testing?](https://www.momentcrm.com/blog/what-is-ab-testing/)
- [5 Common Threats to Your A/B Test’s Validity](https://instapage.com/blog/validating-ab-tests)
- [A/B Testing Statistics: An Easy-to-Understand Guide](https://cxl.com/blog/ab-testing-statistics/)

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 

# Explorando e conhecendo os dados

In [ ]:
dados = pd.read_csv('../input/ab-testing/ab_data.csv')
dados.head()

In [ ]:
dados.shape

In [ ]:
dados.info()

O dataset possui 5 colunas e 294478 linhas.
Nenhum valor nulo. 
A variável Id é numérica, mas é só de identificação e precisa ser verificada se há valores repetidos.
A variável converted é binária e significa se o usuário realizou a ação (1) ou não (0).
O teste é realizado ecom 2 grupos: control e treatment em duas páginas(landing-page)- a nova e a antiga.

## Tratamento dos dados

 
 


## a) Verificando se usuários do grupo de controle viram a nova página e vice versa

No Teste A|B é importante a independência e não interferência entre os grupos, por isso, removeremos os dados dos usuários do grupo de controle que entraram na nova página e vice-versa.

In [ ]:
#separando os usuários de cada grupo que viu cada página.
dados.groupby(['group', 'landing_page']).count()

In [ ]:
#Filtrando somente os usuários que estão de acordo para nossa análise
dados_1 = dados.loc[(dados.group == 'control') & (dados.landing_page == 'old_page')
                   | (dados.group == 'treatment') & (dados.landing_page == 'new_page')]

In [ ]:
#verificando 
dados_1.groupby(['group','landing_page']).count()

## b) Dados repetidos
Verificaremos de há usuários repetidos pelo Id, é comum usuários retornar a página mais de uma vez antes de realizar uma compra, por exemplo, ele pode pesquisar em dispositivos diferentes, mostrar para alguém para aprovação, comparar valores em outros sites e tomar a decisão de compra em outro momento.

In [ ]:
dados_1.shape[0] == dados_1.user_id.nunique()  #False indica que há valores repetidos. Quantos?

In [ ]:
dados_1.shape[0] - dados_1.user_id.nunique()
#significa que há usuários acessaram mais de uma vez.

In [ ]:
# Encontrando o valor duplicado
dados_1[dados_1.duplicated(['user_id'], keep=False)]['user_id']

In [ ]:
#buscando o usuário
dados_1[dados.user_id == 773192]

O usuário do grupo de controle entrou duas vezes e não comprou em nenhuma delas. Para simplificar manteremos somente o primeiro registro.
Note que nossa decisão seria diferente no caso do usuário tivesse comprado.

In [ ]:
dados_1 = dados.drop_duplicates(subset='user_id',keep='first')

# Explorando os dados - EDA

In [ ]:
grupos = dados_1.groupby(['group','landing_page','converted']).size()
grupos

In [ ]:
plt.figure(figsize=(10,8))
grupos.plot.bar(color='grey')



In [ ]:
#verificando se a proporção em cada grupo é similar, isto é, se a amostra está balanceada. 
#Amostras desbalanceadas podem levar a enviesamento dos dados, por isso é importante ter uma representividade semelhante.
dados_1['landing_page'].value_counts().plot(kind='pie', colors=['grey','silver'])

# Taxa de Conversão

### Taxa de conversão geral

In [ ]:
#taxa de conversão
conversao = pd.DataFrame(dados.converted.value_counts()/dados.shape[0] *100)
conversao

In [ ]:
plt.figure(figsize=(10,8))

plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['text.color'] = 'b'
plt.rcParams['font.size']=20

cores = [ 'silver','gray']

percentages = list(conversao.converted)
explode=(0.1,0)
labels = ['Não comprou','Comprou']
plt.pie(percentages, explode=explode, 
       labels=labels,
       colors = cores,
       autopct='%1.0f%%',
       shadow=True, startangle=0,   
       pctdistance=0.5,labeldistance=1.1)
plt.title("Taxa de conversão", fontsize=20, pad=20)


### Taxa de conversão dos grupos
Compareremos a taxa de conversão dos grupos em gráfico de barras.

In [ ]:
conversao_control = pd.DataFrame(dados_1[dados_1.group == 'control']['converted'].value_counts())
conversao_control['percentual'] = conversao_control.converted/ (conversao_control.converted.sum()) *100
conversao_control

In [ ]:
conversao_treat = pd.DataFrame(dados_1[dados_1.group == 'treatment']['converted'].value_counts())
conversao_treat['percentual'] = conversao_treat.converted/ (conversao_treat.converted.sum()) *100
conversao_treat

In [ ]:
fig, (axis1, axis2, axis3) = plt.subplots(1,3, figsize=(30,8))

plt.figure(figsize=(10,8))
ax = sns.barplot(x=conversao_control.index, y="percentual", data=conversao_control, ax=axis1, palette = 'Blues_r')
ax.set_title("Conversão do Grupo de Controle", fontsize=25,pad=20)
ax.set_xlabel('Status', fontsize=15)
ax.set_ylabel('%', fontsize=15)
ax.set_xticklabels(labels=['Não Comprou', 'Comprou'])
plt.tight_layout()

plt.figure(figsize=(10,8))
ax = sns.barplot(x=conversao_treat.index, y="percentual", data=conversao_treat, ax=axis2, palette = 'Greens_r')
ax.set_title("Conversão do Grupo de Experimento", fontsize=25,pad=20)
ax.set_xlabel('Status', fontsize=15)
ax.set_ylabel('%', fontsize=15)
ax.set_xticklabels(labels=['Não Comprou', 'Comprou'])
plt.tight_layout()

plt.figure(figsize=(10,8))
ax = sns.barplot(x=conversao.index, y="converted", data=conversao, ax=axis3, palette = 'Oranges')
ax.set_title("Conversão Geral", fontsize=25,pad=20)
ax.set_xlabel('Status', fontsize=15)
ax.set_ylabel('%', fontsize=15)
ax.set_xticklabels(labels=['Não Comprou', 'Comprou'])
plt.tight_layout()


Visualmente não dá para notar muitas diferenças nos grupos, parecem que se comportam do mesmo modo. E estatísticamente?
Se você leu os artigos recomendados anteriormente, perceberás que esse tipo de situação é mais comum do que se imagina por isso o Teste A|B é tão importante.

# Teste A|B com Chi-quadrado

Chi-Quadrado, também conhecido como coeficiente de Pearson é um teste estatístico para comparar duas variáveis categóricas, independentes entre com pelo menos 5 observações. O teste verifica se as variáveis são homogêneas entre si.

A interpretação é realizada por meio de comparação com o p-value que , por convenção,abaixo de 0,05 há evidências de diferenças entre os grupos e acima disso não há diferenças significativas.

0. *H0* ou Hipótese nula - não há fiferença estatistica entre as variáveis, p > 0,05.
1. *H1* ou Hipótese Alternativa - Há diferença esstatística entre as variáveis, p < 0,05.

Abaixo deixo 1 artigo em português e 3 em inglês para melhor compreensão do assunto. Vale a pena a leitura!

[Teste Chi Quadrado de Pearson: um guia completo](https://sosestatistica.com.br/teste-chi-quadrado-pearson/#:~:text=O%20teste%20Chi%20Quadrado%20de,%C3%972%20(ou%20maiores).)

[Chi-Squared Test for Feature Selection with implementation in Python](https://towardsdatascience.com/chi-squared-test-for-feature-selection-with-implementation-in-python-65b4ae7696db)


[The Chi-Squared Test Statistic is a Must For Every Data Scientist: A Case Study in Customer Churn](https://towardsdatascience.com/the-chi-squared-test-statistic-is-a-must-for-every-data-scientist-a-case-study-in-customer-churn-bcdb17bbafb7)

[Ace the Interview: Chi-Squared Test](https://towardsdatascience.com/ace-the-interview-the-chi-squared-test-e6d37f681ea4)



## Rearranjando os dados em uma tabela 2x2 para aplicarmos o Chi-Quadrado.

In [ ]:
dados_1

In [ ]:
tabela_chi = pd.crosstab(dados_1.landing_page,
                           dados.converted, normalize=False)

tabela_chi

#podemos usar tando a landing_page como o grupo já que eles estão relacionados ao grupo de controle e ao experimento.

## Teste Chi-Quadrado

A função chi_contingence da biblioteca retorna 4 valores: chi_2, p_value, graus de liberdade e frequência esperada.

[Biblioteca](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)

In [ ]:
from scipy.stats import chi2_contingency


In [ ]:
chi_2, p_value, gol,frequencia = scipy.stats.chi2_contingency(tabela_chi,correction=False)

# selecionando o nível de significãncia que por convenção é 0,05
alfa = 0.05
# Aceita ou rejeita H0?
print('significância=%.2f, p_value=%.2f, chi_2=%.2f' % (alfa, p_value,chi_2))
if p_value >= alfa:
    print('Não há diferenças entre as páginas,as variáveis são homogêneas. (aceitamos H0)')
else:
    print('Há indícios de que a conversão nas páginas é diferente, as variáveis são heterogêneas"  (rejeitamos H0 e aceitamos H1)')

Tal resultado significa que não há diferenças estatísticas nas conversões da pagina antiga e a página nova. 